In [1]:
import requests
import csv
import time

In [3]:
checkI = 0 

In [4]:
def storeInfo(as_dict_vulnerability):
    with open('2021.csv', 'a',  newline='') as file:
        writer = csv.writer(file)
        writer.writerow([as_dict_vulnerability['vulnerability_id'], as_dict_vulnerability['reporting_time'],as_dict_vulnerability['vulnerability_description'],
                       as_dict_vulnerability['references'],as_dict_vulnerability['CWE-ID'],as_dict_vulnerability['CWE-Name'],as_dict_vulnerability['CVSS-Score']])

In [5]:
def getVulInfo(urls, urlll):
    global checkI
    as_dict_vulnerability = {
    "vulnerability_id": None, 
    "reporting_time": None, 
    "vulnerability_description": None,
    "references": None,
    "CWE-ID": None, 
    "CWE-Name": None,
    "CVSS-Score": None,
    }
    for i in range(checkI, len(urls)):
        web = requests.get(urls[i], allow_redirects=True)
        raw_data = web.content.decode("UTF-8")
        
        #get id
        id = raw_data.split("<title>NVD - ")
        id = id[1].split("<")[0]
        print(id)
        
        #get date
        date = raw_data.split("vuln-published-on\">")
        date = date[1].split("<")[0]

        #get reference
        reference = raw_data.split("vuln-hyperlinks-row-")
        reference.pop(0)

        links = ""
        n = 2

        for k in reference:
            n -= 1
            href = k.split("href=\"")[1]
            href = href.split("\"")[0]
            links += href
            if (n):
                links += ","

        #get CWE id and name
        try:
            CWE_id = raw_data.split("vuln-CWEs-link-0\">")[1]
            CWE_name = raw_data.split("vuln-CWEs-link-0\">")[2].split("<")[0]

            if (len(CWE_id.split("href=\"")) > 1):
                CWE_id = CWE_id.split("href=\"")[1].split("\"")[0]
            else:
                CWE_id = CWE_id.split("<span>")[1].split("<")[0]
        except Exception as e: 
            CWE_id = "Unavailable"
            CWE_name = "Unavailable"

        #get CVSS_score
        CVSS_score = raw_data.split("vuln-cvss3-panel-score")
            
        if (len(CVSS_score) > 1):
            CVSS_score = raw_data.split("vuln-cvss3-panel-score")[1]
            CVSS_score = CVSS_score.split(">")[1].split(" ")[0]
        else:
            CVSS_score = "N/A"
        print(CVSS_score)
        #get description
        description = raw_data.split("description\">")
        description = description[1].split("<")[0]
        
         #fill dictionary
        as_dict_vulnerability.update({"vulnerability_id": id})
        as_dict_vulnerability.update({"reporting_time": date})
        as_dict_vulnerability.update({"vulnerability_description": description})
        as_dict_vulnerability.update({"references": links})
        as_dict_vulnerability.update({"CWE-ID": CWE_id})
        as_dict_vulnerability.update({"CWE-Name": CWE_name})
        as_dict_vulnerability.update({"CVSS-Score": CVSS_score})
        storeInfo(as_dict_vulnerability)
        
        if (checkI % 50 == 0 and checkI > 0):
            print(checkI)
        checkI += 1
        
    checkI = 0

In [6]:
def getVulnerabilities():
    Year = 2021
    dict_arr_vulnerability = []
    global checkI
    checkj = 0
    j = 1

    while j <= 12:
        try:
            if checkj != j:
                checkj = j
                urlll = 'https://nvd.nist.gov/vuln/full-listing/{year}/{tag}'
                urlll = urlll.format(year = Year, tag = j)
                print(urlll)
                r = requests.get(urlll, allow_redirects=True)
                raw = r.content.decode("UTF-8")
                step1 = raw.split("/vuln/detail/CVE-") #list of CVE links eg: ("<a href="/vuln/detail/CVE-2012-10001">CVE-2012-10001</a>")
                step2 = [] #list of CVE ids
                for i in step1:
                    step2.append(i.split("\">")[0]) #append just CVE id into step2

                step2.pop(0) #remove <html lang="en
                urls = []

                #create CVE urls
                for i in step2:
                    url = "https://nvd.nist.gov/vuln/detail/CVE-{tag}"
                    url = url.format(tag = i)
                    urls.append(url)
                print("Number of vulnerabilities left: %d"%(len(urls) - checkI))
                getVulInfo(urls, urlll)
                    
        except Exception as e: 
            print("An Error has Occured")
            print(e)
#             j -= 1
#             print("\nFailed at checkI: %d"%(checkI))
#             print("\nSkipped url: " + urls[checkI] + "\n")
#             checkI += 1

#             print("Program will sleep for 10 seconds")
#             time.sleep(10)
#             print("Program is operational again")

        j += 1
        
    
    print("Program Complete")

In [7]:
getVulnerabilities()

https://nvd.nist.gov/vuln/full-listing/2021/1
Number of vulnerabilities left: 1936
CVE-2020-35391
6.5
CVE-2020-35717
9.0
CVE-2021-3002
6.1
CVE-2020-28851
7.5
CVE-2020-28852
7.5
CVE-2020-35952
6.5
CVE-2021-3004
7.5
CVE-2021-3005
4.3
CVE-2020-28841
5.5
CVE-2021-3006
7.5
CVE-2020-35962
7.5
CVE-2020-35963
7.8
CVE-2020-35964
6.5
CVE-2020-35965
7.5
CVE-2021-21494
4.8
CVE-2021-21495
8.8
CVE-2021-3007
9.8
CVE-2019-16956
5.4
CVE-2019-16960
5.4
CVE-2020-28464
N/A
CVE-2020-7771
9.8
CVE-2020-4909
4.8
CVE-2020-4910
4.8
CVE-2020-4912
7.2
CVE-2020-4913
4.4
CVE-2020-4916
4.8
CVE-2020-4917
8.8
CVE-2020-4918
4.4
CVE-2020-4919
3.8
CVE-2020-4928
6.7
CVE-2020-4942
8.8
CVE-2020-22550
7.5
CVE-2020-35493
5.5
CVE-2020-35494
6.1
CVE-2020-35495
5.5
CVE-2020-35496
5.5
CVE-2020-35507
5.5
CVE-2020-36112
9.8
CVE-2020-17535
N/A</a
CVE-2020-17536
N/A</a
CVE-2020-17537
N/A</a
CVE-2020-24386
6.8
CVE-2020-25275
7.5
CVE-2020-36154
7.8
CVE-2019-25013
5.9
CVE-2020-26292
9.8
CVE-2020-35219
9.8
CVE-2020-36155
9.8
CVE-2020-361

CVE-2020-16040
6.5
CVE-2020-16041
8.1
CVE-2020-16042
6.5
CVE-2020-16043
8.8
CVE-2020-26186
N/A
CVE-2020-5017
5.5
CVE-2020-5018
7.5
CVE-2020-5019
6.5
CVE-2020-5020
6.1
CVE-2020-5021
4.4
CVE-2020-5022
5.3
CVE-2021-21106
9.6
CVE-2021-21107
9.6
CVE-2021-21108
9.6
CVE-2021-21109
9.6
CVE-2021-21110
9.6
CVE-2021-21111
9.6
CVE-2021-21112
8.8
CVE-2021-21113
8.8
CVE-2021-21114
8.8
CVE-2021-21115
9.6
CVE-2021-21116
8.8
CVE-2020-4487
4.3
CVE-2020-4544
4.3
CVE-2020-4691
5.4
CVE-2020-4697
5.4
CVE-2020-4733
5.4
CVE-2020-5146
7.2
CVE-2020-5147
5.3
CVE-2020-35203
6.1
CVE-2020-35204
6.1
CVE-2020-35205
9.8
CVE-2020-35206
6.1
CVE-2020-35719
6.1
CVE-2020-35720
5.4
450
CVE-2020-35721
5.4
CVE-2020-35722
6.5
CVE-2020-35723
5.4
CVE-2020-35724
5.4
CVE-2020-35725
6.1
CVE-2020-35726
6.1
CVE-2020-35727
5.4
CVE-2021-3116
7.5
CVE-2021-3118
9.8
CVE-2021-3121
8.6
CVE-2020-11995
9.8
CVE-2020-13922
6.5
CVE-2020-17508
7.5
CVE-2020-17509
7.5
CVE-2020-26800
5.5
CVE-2020-23643
6.1
CVE-2020-23644
6.1
CVE-2020-23849
6.1
CVE-2

CVE-2020-27488
9.8
CVE-2020-9145
9.1
CVE-2020-35578
7.2
CVE-2021-1189
7.2
CVE-2020-9138
5.3
CVE-2020-9139
9.1
850
CVE-2020-9140
9.8
CVE-2020-9141
9.1
CVE-2020-9142
9.1
CVE-2020-9143
5.3
CVE-2020-9203
3.3
CVE-2021-1126
5.5
CVE-2021-1127
5.4
CVE-2021-1130
4.8
CVE-2021-1131
4.3
CVE-2021-1143
4.3
CVE-2021-1144
8.8
CVE-2021-1145
6.5
CVE-2021-1146
7.2
CVE-2021-1147
7.2
CVE-2021-1148
7.2
CVE-2021-1149
7.2
CVE-2021-1150
7.2
CVE-2021-1151
4.8
CVE-2021-1152
4.8
CVE-2021-1153
4.8
CVE-2021-1154
4.8
CVE-2021-1155
4.8
CVE-2021-1156
4.8
CVE-2021-1157
4.8
CVE-2021-1158
4.8
CVE-2021-1159
7.2
CVE-2021-1160
7.2
CVE-2021-1161
7.2
CVE-2021-1162
7.2
CVE-2021-1163
7.2
CVE-2021-1164
7.2
CVE-2021-1165
7.2
CVE-2021-1166
7.2
CVE-2021-1167
7.2
CVE-2021-1168
7.2
CVE-2021-1169
7.2
CVE-2021-1170
7.2
CVE-2021-1171
7.2
CVE-2021-1172
7.2
CVE-2021-1173
7.2
CVE-2021-1174
7.2
CVE-2021-1175
7.2
CVE-2021-1176
7.2
CVE-2021-1177
7.2
CVE-2021-1178
7.2
CVE-2021-1179
7.2
CVE-2021-1180
7.2
CVE-2021-1181
7.2
CVE-2021-1182
7.2
CVE-

CVE-2020-27858
7.5
CVE-2020-27859
7.5
CVE-2021-1129
5.3
CVE-2021-1133
7.3
CVE-2021-1253
5.4
CVE-2021-1255
5.4
CVE-2021-1257
8.8
CVE-2021-1259
6.5
CVE-2021-1260
7.8
CVE-2021-1261
7.8
CVE-2021-1262
7.8
CVE-2021-1263
7.8
1300
CVE-2021-1264
8.8
CVE-2021-1265
6.5
CVE-2021-1269
6.3
CVE-2021-1270
6.5
CVE-2021-1271
4.8
CVE-2021-1272
8.8
CVE-2021-1273
8.6
CVE-2021-1274
8.6
CVE-2021-1276
6.5
CVE-2021-1277
6.5
CVE-2021-1278
7.5
CVE-2021-1279
8.6
CVE-2021-1280
7.3
CVE-2021-1282
4.9
CVE-2021-1283
5.5
CVE-2021-1286
6.1
CVE-2021-1298
8.8
CVE-2021-1299
8.8
CVE-2021-1300
9.8
CVE-2021-1301
9.8
CVE-2021-1302
8.8
CVE-2021-1303
8.8
CVE-2021-1304
6.5
CVE-2021-1305
4.3
CVE-2021-1312
7.5
CVE-2021-1349
6.5
CVE-2021-1350
5.3
CVE-2021-1353
8.6
CVE-2021-1355
6.5
CVE-2021-1357
6.5
CVE-2021-1364
4.9
CVE-2021-1135
4.3
CVE-2021-1138
9.8
CVE-2021-1139
8.8
CVE-2021-1140
9.8
CVE-2021-1141
8.8
CVE-2021-1142
9.8
CVE-2021-1218
5.4
CVE-2021-1219
7.8
CVE-2021-1222
8.1
CVE-2021-1225
9.1
CVE-2021-1233
4.4
CVE-2021-1235
5.5
CVE

CVE-2020-5489
N/A</a
CVE-2020-5490
N/A</a
CVE-2020-5491
N/A</a
CVE-2020-5492
N/A</a
CVE-2020-5493
N/A</a
1700
CVE-2020-5494
N/A</a
CVE-2020-5495
N/A</a
CVE-2020-6779
N/A
CVE-2020-6780
4.9
CVE-2020-8288
5.4
CVE-2020-8292
5.4
CVE-2020-8293
6.5
CVE-2020-8295
7.5
CVE-2020-9492
8.8
CVE-2021-21615
5.3
CVE-2021-21723
7.5
CVE-2021-22697
7.8
CVE-2021-22698
7.8
CVE-2021-22871
4.8
CVE-2021-22872
6.1
CVE-2021-22873
6.1
CVE-2021-25863
8.8
CVE-2021-25864
7.5
CVE-2021-25900
9.8
CVE-2021-25901
5.3
CVE-2021-25902
7.5
CVE-2021-25903
7.5
CVE-2021-25904
7.5
CVE-2021-25905
9.1
CVE-2021-25906
7.5
CVE-2021-25907
9.8
CVE-2021-25908
7.5
CVE-2021-26025
7.8
CVE-2021-26026
7.8
CVE-2021-26266
7.5
CVE-2021-26267
7.5
CVE-2021-3114
6.5
CVE-2021-3115
7.5
CVE-2021-3152
5.3
CVE-2021-3164
8.8
CVE-2021-3185
9.8
CVE-2021-3186
6.1
CVE-2021-3188
9.8
CVE-2021-3190
9.8
CVE-2021-3193
9.8
CVE-2021-3195
7.5
CVE-2021-3199
9.8
CVE-2021-3223
7.5
CVE-2021-3278
9.8
CVE-2021-3285
N/A
CVE-2021-3286
9.8
CVE-2021-3291
7.2
CVE-2021-3297
7.

...

CVE-2021-24855
5.4
CVE-2021-24857
9.8
CVE-2021-24859
4.3
CVE-2021-24861
7.2
CVE-2021-24863
9.8
CVE-2021-24871
5.4
CVE-2021-24872
6.5
CVE-2021-24896
4.8
CVE-2021-24922
9.0
CVE-2021-24925
6.1
CVE-2021-24932
6.1
CVE-2021-24945
8.0
CVE-2021-24946
9.8
CVE-2021-24951
9.8
CVE-2021-24954
6.1
CVE-2021-24955
6.1
CVE-2021-24970
7.2
CVE-2021-24972
4.8
CVE-2021-42546
6.1
CVE-2021-42547
6.1
CVE-2021-42548
6.1
CVE-2021-42549
6.1
600
CVE-2021-43117
9.8
CVE-2021-36169
6.0
CVE-2021-44965
7.5
CVE-2021-44966
9.8
CVE-2021-22279
N/A
CVE-2021-39910
4.3
CVE-2021-39915
N/A
CVE-2021-39916
N/A
CVE-2021-39917
6.5
CVE-2021-39918
4.3
CVE-2021-39919
N/A
CVE-2021-39930
N/A
CVE-2021-39931
4.3
CVE-2021-39932
N/A
CVE-2021-39933
6.5
CVE-2021-39934
N/A
CVE-2021-39935
7.5
CVE-2021-39936
4.3
CVE-2021-39937
8.8
CVE-2021-39938
6.5
CVE-2021-39939
N/A
CVE-2021-39940
6.5
CVE-2021-39941
5.3
CVE-2021-39944
N/A
CVE-2021-39945
N/A
CVE-2021-40007
6.5
CVE-2021-40008
7.5
CVE-2021-43983
N/A
CVE-2020-16154
7.8
CVE-2020-16155
6.5
CVE-2020

CVE-2021-45078
7.8
CVE-2021-4119
9.8
CVE-2021-44116
6.1
CVE-2020-18984
6.1
CVE-2020-18985
6.1
CVE-2021-44350
9.8
CVE-2021-45017
8.8
CVE-2021-45018
6.1
CVE-2021-43833
8.8
CVE-2021-43834
9.8
CVE-2021-44023
7.1
CVE-2021-45085
6.1
CVE-2021-45086
6.1
CVE-2021-45087
6.1
CVE-2021-45088
6.1
CVE-2021-45092
9.8
CVE-2021-45095
5.5
CVE-2021-45096
4.3
CVE-2021-45097
5.5
CVE-2021-45098
N/A</a
CVE-2021-45099
N/A</a
CVE-2021-45100
7.5
CVE-2021-45101
8.1
CVE-2021-45102
8.8
CVE-2021-4121
6.1
CVE-2021-4123
6.5
CVE-2021-40835
N/A</a
CVE-2021-4124
6.1
CVE-2021-3959
7.5
CVE-2021-3960
7.8
CVE-2021-42912
8.8
CVE-2021-41260
8.8
CVE-2021-41962
4.8
CVE-2021-37262
7.5
CVE-2021-38244
7.5
CVE-2021-41028
7.5
1050
CVE-2021-41261
4.8
CVE-2021-41262
N/A
CVE-2021-42550
N/A
CVE-2021-43812
6.1
CVE-2021-43837
N/A
CVE-2021-44315
7.5
CVE-2021-44317
5.4
CVE-2020-35209
7.5
CVE-2020-35210
6.5
CVE-2020-35211
7.5
CVE-2020-35213
8.1
CVE-2020-35214
8.1
CVE-2020-35215
6.5
CVE-2020-35216
5.9
CVE-2021-26800
6.5
CVE-2021-3179
5.5
CVE-2

CVE-2020-11376
N/A</a
CVE-2020-11377
N/A</a
CVE-2020-11378
N/A</a
CVE-2020-11379
N/A</a
CVE-2020-11380
N/A</a
CVE-2020-11381
N/A</a
CVE-2020-11382
N/A</a
CVE-2020-11383
N/A</a
CVE-2020-11384
N/A</a
CVE-2020-11385
N/A</a
CVE-2020-11386
N/A</a
CVE-2020-11387
N/A</a
CVE-2020-11388
N/A</a
CVE-2020-11389
N/A</a
CVE-2020-11390
N/A</a
CVE-2020-11391
N/A</a
CVE-2020-11392
N/A</a
CVE-2020-11393
N/A</a
CVE-2020-11394
N/A</a
CVE-2020-11395
N/A</a
CVE-2020-11396
N/A</a
CVE-2020-11397
N/A</a
CVE-2020-11398
N/A</a
CVE-2020-11399
N/A</a
CVE-2020-11400
N/A</a
CVE-2020-11401
N/A</a
CVE-2020-11402
N/A</a
CVE-2020-11403
N/A</a
CVE-2020-11404
N/A</a
CVE-2020-11405
N/A</a
CVE-2020-11406
N/A</a
CVE-2020-11407
N/A</a
CVE-2020-11408
N/A</a
CVE-2020-11409
N/A</a
CVE-2020-11410
N/A</a
CVE-2020-11411
N/A</a
CVE-2020-11412
N/A</a
CVE-2020-11413
N/A</a
CVE-2020-16637
N/A</a
CVE-2020-16638
N/A</a
1450
CVE-2020-16639
N/A</a
CVE-2020-16640
N/A</a
CVE-2020-16641
N/A</a
CVE-2020-16642
N/A</a
CVE-2020-16643
N/A</a
CVE-2

CVE-2021-21907
N/A</a
CVE-2021-21908
N/A</a
CVE-2021-21909
N/A</a
CVE-2021-21910
7.8
CVE-2021-21911
7.8
CVE-2021-21912
7.8
CVE-2021-21915
8.8
CVE-2021-21916
9.8
CVE-2021-21917
8.8
CVE-2021-21918
8.8
1800
CVE-2021-21919
8.8
CVE-2021-21920
8.8
CVE-2021-21921
8.8
CVE-2021-21922
8.8
CVE-2021-21923
8.8
CVE-2021-21924
8.8
CVE-2021-21925
8.8
CVE-2021-21926
6.5
CVE-2021-21927
8.8
CVE-2021-21928
8.8
CVE-2021-21929
8.8
CVE-2021-21930
8.8
CVE-2021-21931
8.8
CVE-2021-21932
8.8
CVE-2021-21933
8.8
CVE-2021-21934
8.8
CVE-2021-21935
8.8
CVE-2021-21936
8.8
CVE-2021-21937
8.8
CVE-2021-21952
N/A</a
CVE-2021-21953
N/A</a
CVE-2021-23228
6.1
CVE-2021-31558
6.1
CVE-2021-36885
N/A
CVE-2021-36886
8.8
CVE-2021-39306
N/A</a
CVE-2021-40393
N/A</a
CVE-2021-40394
N/A</a
CVE-2021-40417
N/A</a
CVE-2021-40418
N/A</a
CVE-2021-44471
6.1
CVE-2021-44544
6.1
CVE-2021-45461
N/A</a
CVE-2021-43853
N/A
CVE-2020-20425
6.1
CVE-2020-20426
6.1
CVE-2020-20593
8.0
CVE-2020-20595
6.5
CVE-2020-20597
6.1
CVE-2020-20598
6.1
CVE-2020-206

CVE-2021-44598
N/A</a
CVE-2021-45712
N/A</a
CVE-2021-45713
N/A</a
CVE-2021-45714
N/A</a
CVE-2021-45715
N/A</a
CVE-2021-45716
N/A</a
CVE-2021-45717
N/A</a
CVE-2021-45718
N/A</a
CVE-2021-45719
N/A</a
CVE-2021-45720
N/A</a
CVE-2018-25023
N/A</a
CVE-2018-25024
N/A</a
CVE-2018-25025
N/A</a
CVE-2018-25026
N/A</a
CVE-2018-25027
N/A</a
CVE-2018-25028
N/A</a
CVE-2019-25054
N/A</a
CVE-2019-25055
N/A</a
CVE-2020-36511
N/A</a
CVE-2020-36512
N/A</a
CVE-2020-36513
N/A</a
CVE-2020-36514
N/A</a
CVE-2021-45680
N/A</a
CVE-2021-45681
N/A</a
CVE-2021-45682
N/A</a
CVE-2021-45683
N/A</a
2200
CVE-2021-45684
N/A</a
CVE-2021-45685
N/A</a
CVE-2021-45686
N/A</a
CVE-2021-45687
N/A</a
CVE-2021-45688
N/A</a
CVE-2021-45689
N/A</a
CVE-2021-45690
N/A</a
CVE-2021-45691
N/A</a
CVE-2021-45692
N/A</a
CVE-2021-45693
N/A</a
CVE-2021-45694
N/A</a
CVE-2021-45695
N/A</a
CVE-2021-45696
N/A</a
CVE-2021-45697
N/A</a
CVE-2021-45698
N/A</a
CVE-2021-45699
N/A</a
CVE-2021-45700
N/A</a
CVE-2021-45701
N/A</a
CVE-2021-45702
N/A</a
CVE-2

In [26]:
def analyzeData(fileName):
    count = 0
    with open(fileName, mode='r', encoding='utf-8-sig') as f:
        vuls = csv.DictReader(f)
        for row in vuls:
            if "** REJECT **" in row['vulnerability_description']:
                if "** REJECT **  DO NOT USE THIS CANDIDATE NUMBER." in row['vulnerability_description']:
                    print(row['vulnerability_description'])
#                     "** REJECT ** DO NOT USE THIS CANDIDATE NUMBER"
                count+=1
    print(count)
    print(132270-count)

In [27]:
analyzeData("NVD_2011-2021.csv")

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any issues during 2010.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2009-1012.  Reason: This candidate is a reservation duplicate of CVE-2009-1012.  Notes: All CVE users should reference CVE-2009-1012 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2009-1016.  Reason: This candidate is a reservation duplicate of CVE-2009-1016.  Notes: All CVE users should reference CVE-2009-1016 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2007-1685.  Reason: This candidate is a duplicate of CVE-2007-1685.  Notes: All CVE users should reference CVE-2007-1

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2013-1636.  Reason: This candidate is a reservation duplicate of CVE-2013-1636.  Notes: All CVE users should reference CVE-2013-1636 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: CVE-2012-5217.  Reason: This candidate is a duplicate of CVE-2012-5217.  A typo caused the wrong ID to be used.  Notes: All CVE users should reference CVE-2012-5217 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2013-4125.  Reason: This candidate is a reservation duplicate of CVE-2013-4125.  Notes: All CVE users should reference CVE-2013-4125 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** R

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any issues during 2014.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any issues during 2014.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was withdrawn by its CNA.  Further investigation showed that it was not a security issue in customer-controlled software.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2014-3570.  Reason: This candidate is a reservation duplicate of CVE-2014-3570.  Notes: All CVE users should reference CVE-2014-3570 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was withdrawn b

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: CVE-2016-6277. Reason: This candidate was withdrawn by its CNA. Notes: All CVE users should reference CVE-2016-6277 instead of this candidate. All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2016-8823.  Reason: This candidate is a reservation duplicate of CVE-2016-8823.  Notes: All CVE users should reference CVE-2016-8823 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2016-4552.  Reason: This candidate is a reservation duplicate of CVE-2016-4552.  Notes: All CVE users should reference CVE-2016-4552 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  Consu

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2012. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2013. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2013. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2013. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2013. Notes: none.
** REJECT **  DO NOT USE 

...

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was withdrawn by its CNA.  Further investigation showed that it was not a security issue associated with the originally named downstream provider.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was withdrawn by its CNA.  Further investigation showed that it was not a security issue associated with the originally named downstream provider.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was withdrawn by its CNA.  Further investigation showed that it was not a security issue associated with the originally named downstream provider.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was withdrawn by its CNA.  Further investigation showed that it was not a security issue associated with the originally named downstream provider.  Notes

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any issues during 2017.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any issues during 2017.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any issues during 2017.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any issues during 2017.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any issues during 2017.  Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: none.  Reason: This candidate was in a CNA pool that was not assigned to any is

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER.  ConsultIDs: CVE-2018-15727. Reason: This candidate is a reservation duplicate of CVE-2018-15727. Notes: All CVE users should reference CVE-2018-15727 instead of this candidate.  All references and descriptions in this candidate have been removed to prevent accidental usage.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2017. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2017. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2017. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or indivi

** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2017. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2018. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2018. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2017. Notes: none.
** REJECT **  DO NOT USE THIS CANDIDATE NUMBER. ConsultIDs: none. Reason: The CNA or individual who requested this candidate did not associate it with any vulnerability during 2018. Notes: none.
** REJECT **  DO NOT USE 

9926
122344
